In [ ]:
# поработаем с адресами.
# почти каждая строка имеет уникальный адрес, что неудобно для работы

In [2]:
import pandas as pd
from IPython.display import display

melb_data = pd.read_csv('data11/melb_data.csv', sep = ',')
melb_df = melb_data.copy()

print(melb_df['Address'].nunique()) # метод нахождения количества
# уникальных значений в столбце

13378


In [5]:
# извлечем несколько адресов, чтобы понять, как именно они записаны
print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])

2/119 Railway St N
9/400 Dandenong Rd
172 Danks St


## Вывод: сначала указывается номер дома и корпус, после указывается название улицы, а в конце — подтип улицы, но в некоторых случаях к подтипу добавляется географическая отметка (N — север, S — юг и т. д.), она нам не нужна . Для того чтобы выделить подтип улицы, на которой находится объект, можно использовать следующую функцию:

In [6]:
def get_street_type(address): # принимает адрес
    exclude_list = ['N', 'S', 'W', 'E'] # список геогрфических отметок
    address_list = address.split() # разбили строку с адресом на слова по пробелу,
    # занесли этот список в переменную
    street_type = address_list[-1] # берем только последний элемент из адреса
    
    if street_type in exclude_list: # если посл элемент - геогр отметка
        street_type = address_list[-2] # берем предпоследний элемент
    return street_type

street_types = melb_df['Address'].apply(get_street_type) # применили функцию 
# ко всему столбцу
display(street_types)

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

In [33]:
street_types = street_types.apply(lambda x: 'Av' if x == 'Avenue' else x)
street_types = street_types.apply(lambda x: 'Bvd' if x == 'Boulevard' else x)
street_types = street_types.apply(lambda x: 'Pde' if x == 'Parade' else x)

In [34]:
print(street_types.nunique()) # сколько теперь уникальных значений

53


In [35]:
display(street_types.value_counts()) # частота этих уникальных значений

St           8012
Rd           2825
Ct            612
Dr            447
Av            361
Gr            311
Pde           226
Pl            169
Cr            152
Cl            100
La             67
Bvd            66
Tce            47
Wy             40
Cct            25
Hwy            24
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Mews            4
Grn             4
Fairway         4
Gdns            4
Righi           3
Crossway        3
Esp             2
Victoria        2
Ridge           2
Crofts          2
Grand           1
Summit          1
Hts             1
Athol           1
Highway         1
Outlook         1
Woodland        1
Ave             1
Gra             1
Terrace         1
Eyrie           1
Dell            1
East            1
Loop            1
Nook            1
Glade           1
Qy              1
Cove            1
Res             1
Grange          1
Corso           1
Name: Address, dtype: int64

In [36]:
popular_stypes = street_types.value_counts().nlargest(10).index
# выделим 10 самых популярных значений
# атрибут index => список названий
display(popular_stypes)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')

In [37]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
# создали новый столбец StreetTypes
# lambda проверяет, какие значения из 10 популярных, если нет - они 'other'
# результат lambda применяется ко всему списку StreetTypes
# и заносится в новый столбец 
display(melb_df['StreetType'])

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

In [47]:
print(melb_df['StreetType'].nunique())

11


In [41]:
melb_df = melb_df.drop(['Address'], axis = 1) # удаляем старый столбец

KeyError: "['Address'] not found in axis"

# В таблице остались подтипы, которые именуются различным образом, но при этом обозначают одинаковые вещи. Например, подтипы Av и Avenue, Bvd и Boulevard, Pde и Parade. Преобразуем полные названия в топографические сокращения

In [ ]:
(lambda x: 'Av' if x == 'Avenue' else x) # для каждого названия
# применить надо до сокращения уникальных значений (ячейка 7)

Задание 1

# Ранее, в задании 3.3, мы создали признак WeekdaySale в таблице melb_df — день недели продажи. Из полученных в задании результатов можно сделать вывод, что объекты недвижимости в Мельбурне продаются преимущественно по выходным (суббота и воскресенье).
# Напишите функцию get_weekend(weekday), которая принимает на вход элемент столбца WeekdaySale и возвращает 1, если день является выходным, и 0 — в противном случае, и создайте столбец Weekend в таблице melb_df с помощью неё.
# Примените эту функцию к столбцу и вычислите среднюю цену объекта недвижимости, проданного в выходные дни. Результат округлите до целых.

In [63]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek

def get_weekend(weekday): 
    if weekday == 5 or weekday == 6:
        return 1
    else: 
        return 0

melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
mean_weekend_price = melb_df[melb_df["Weekend"]==1]["Price"].mean()
print(mean_weekend_price)

1081198.6406956792


Задание 2 

# Преобразуйте столбец SellerG с наименованиями риелторских компаний в таблице melb_df следующим образом: оставьте в столбце только 49 самых популярных компаний, а остальные обозначьте как 'other'.
# Найдите, во сколько раз минимальная цена объектов недвижимости, проданных компанией 'Nelson', больше минимальной цены объектов, проданных компаниями, обозначенными как 'other'. Ответ округлите до десятых.

In [72]:
popular = melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular else 'other')
N_price = melb_df[melb_df["SellerG"]=='Nelson']['Price'].min()
other_price = melb_df[melb_df["SellerG"]=='other']['Price'].min()
print(N_price/other_price)

1.297709923664122


Задание 3

# Представьте, что вы занимаетесь подготовкой данных о вакансиях с платформы hh.ru. В вашем распоряжении имеется таблица, в которой с помощью парсинга собраны резюме кандидатов. В этой таблице есть текстовый столбец «Опыт работы». Пример такого столбца представлен ниже в виде объекта Series. Структура текста в столбце фиксирована и не может измениться.
# Напишите функцию get_experience(arg), аргументом которой является строка столбца с опытом работы. Функция должна возвращать опыт работы в месяцах. Не забудьте привести результат к целому числу.
# Обратите внимание, что опыт работы может выражаться только в годах или только в месяцах. Учтите это при построении своей функции. Кроме того, учтите возможные вариации слов месяц (месяца, месяцев) и год (года, лет).

In [5]:
import pandas as pd

def get_experience(arg):
    month_name = ['месяц', 'месяцев', 'месяца']
    year_name = ['год', 'года', 'лет']
    month = 0
    year = 0
    args_splited = arg.split(' ')
    
    for i in range(len(args_splited)):
        if args_splited[i] in year_name:
            year = args_splited[i-1]
        elif args_splited[i] in month_name:
            month = args_splited[i-1]
    return int(year)*12 + int(month)

if __name__ == '__main__':
    experience_col = pd.Series([
        'Опыт работы 8 лет 3 месяца',
        'Опыт работы 3 года 5 месяцев',
        'Опыт работы 1 год 9 месяцев',
        'Опыт работы 3 месяца',
        'Опыт работы 6 лет'
        ])
    experience_month = experience_col.apply(get_experience)
    print(experience_month)

0    99
1    41
2    21
3     3
4    72
dtype: int64
